In [110]:
import pybaseball
import pickle
import gzip
import pandas as pd

In [17]:
data = pybaseball.statcast('2018-01-01','2024-12-31',team='TB')

This is a large query, it may take a moment to complete


C:\Users\taeh0221\AppData\Local\Programs\Python\Python313\Lib\site-packages\pybaseball\statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)


Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates


C:\Users\taeh0221\AppData\Local\Programs\Python\Python313\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
C:\Users\taeh0221\AppData\Local\Programs\Python\Python313\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
C:\Users\taeh0221\AppData\Local\Programs\Python\Python313\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `e

In [98]:
tb_home = data[data['home_team']=='TB']

In [99]:
tb_home['Season'] = tb_home['game_date'].dt.year

C:\Users\taeh0221\AppData\Local\Temp\ipykernel_5236\3371117846.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tb_home['Season'] = tb_home['game_date'].dt.year


In [100]:
batting_stat = pybaseball.batting_stats(2018,2024,qual=50)
fg_id_df = pybaseball.playerid_reverse_lookup(batting_stat['IDfg'].unique(),key_type='fangraphs')

In [103]:
id_dict = {}
for key,val in fg_id_df[['key_fangraphs','key_mlbam']].values:
    id_dict[int(key)] = int(val)

In [104]:
batting_stat['batter'] = batting_stat['IDfg'].map(id_dict)

In [106]:
gb_data = batting_stat[['Season','batter','GB%','FB%','IFFB%','HR/FB']]
gb_data.head()

,Season,batter,GB%,FB%,IFFB%,HR/FB
1,2024,592450.0,0.305,0.462,0.056,0.322
4,2022,592450.0,0.373,0.435,0.046,0.356
59,2024,677951.0,0.370,0.444,0.109,0.134
7,2018,605141.0,0.339,0.449,0.092,0.164
9,2018,545361.0,0.313,0.453,0.094,0.245


In [112]:
tb_home_merged = pd.merge(tb_home,gb_data,on=['Season','batter'],how='inner')

In [113]:
# save and compress.
with gzip.open('tb_data_merged.pkl', 'wb') as f:
    pickle.dump(tb_home_merged, f)

# # load and uncompress.
# with gzip.open('testPickleFile.pickle','rb') as f:
#     data = pickle.load(f)
